In [68]:
import pandas as pd
from datetime import datetime
import requests
from bs4 import BeautifulSoup
from postgre_cred import database_url
from sqlalchemy import create_engine
import json
import random

In [93]:
with open('links.json', 'r') as f:
    urls = json.load(f)
iplus = urls['iplus']
for db_name, url in iplus.items():
    try:
        
        items = get_items(url)
        data = get_data(items)
        engine = create_engine(database_url)
        df = pd.DataFrame(data)
        df = clean_data(df)
        df.to_sql(db_name, con=engine, if_exists='replace', index=True)
        print(f'Successfully exported {db_name}.')
    except Exception as e:
        print(f'Error occured on {db_name}')


 https://iplus.com.ge/ka/iphone-ka/?items_per_page=
 200

Error processing a product: 'NoneType' object is not subscriptable
Successfully exported iplus_iphone.

 https://iplus.com.ge/ka/mac/?items_per_page=
 200

Error processing a product: 'NoneType' object has no attribute 'text'
Error processing a product: 'NoneType' object has no attribute 'text'
Successfully exported iplus_mac.

 https://iplus.com.ge/ka/ipad/?items_per_page=
 200

Error processing a product: 'NoneType' object has no attribute 'text'
Error processing a product: 'NoneType' object has no attribute 'text'
Successfully exported iplus_ipad.

 https://iplus.com.ge/ka/watch/?items_per_page=
 200

Error processing a product: 'NoneType' object has no attribute 'text'
Error processing a product: 'NoneType' object has no attribute 'text'
Successfully exported iplus_watch.

 https://iplus.com.ge/ka/console/?items_per_page=
 200

Error processing a product: 'NoneType' object has no attribute 'text'
Error processing a product:

In [92]:
def clean_data(df: pd.DataFrame):
    df.loc[:, 'current_price'] = df['current_price'].astype(float)
    df.loc[:, 'old_price'] = df['old_price'].astype(float)
    df['update_date'] = datetime.now()
    return df

In [56]:
def get_items(url):
    resp = requests.get(f'{url}{random.randint(4444, 5555)}')
    print(f'\n {url}\n {resp.status_code}\n')
    soup = BeautifulSoup(resp.text, 'html.parser')
    items = soup.select('#categories_view_pagination_contents > div')
    return items

In [41]:

def get_data(items) :
    products = []
    for product in items:
        try:
            name = product.select_one(".ut2-gl__name a").text.strip()
            product_url = product.select_one(".ut2-gl__name a")["href"]
            image = product.select_one(".ut2-gl__image img")["data-src"]
    
            # Handle old price (may be missing)
            old_price_tag = product.select_one(".ty-strike .ty-list-price")  # Select only strikethrough price
            old_price = old_price_tag.text.strip() if old_price_tag else None  
    
            # Handle current price (may be missing)
            current_price_tag = product.select_one(".ty-price-num")
            current_price = current_price_tag.text.strip() if current_price_tag else None  
    
            products.append({
                "name": name,
                "product_url": product_url,
                "image": image,
                "old_price": old_price,
                "current_price": current_price
            })
        
        except Exception as e:
            print(f"Error processing a product: {e}")
    return products